In [34]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
from diffpy.pdfgui.tui import LoadProject
from diffpy.pdfgui.control.pdfguicontrol import PDFGuiControl
from diffpy.pdffit2.pdffit2 import calculationError

from IPython.display import clear_output
# from tqdm.auto import tqdm
import time

In [35]:
%matplotlib widget

In [36]:
def tqdm_sleep_sec(t, message='Sleep'):
    import datetime
    from tqdm import tqdm
    now = datetime.datetime.now()
    print(f'\nSleep for a while: {t} seconds.')
    print(f'Waiting starts at {now}')
    for i in tqdm(range(0,100), desc=message):
        time.sleep(t/100)

## Load a pre-defined PDFGui project file (.ddp) using diffpy.pdfgui.tui.LoadProject

In [37]:
fn = '/Users/cheng-hunglin/Documents/Data_LDRD/Cs_x-Pb_y-B_z_04_02.ddp3'
ddp = LoadProject(fn)

## Check the fitting object (diffpy.pdfgui.control.fitting.Fitting) in the ddp project file

In [38]:
fit_demo = ddp.getFits()[4]
fit_demo.name

'Fit 2_01'

In [39]:
fit_demo.owner

## Create an object of PDFGuiControl which holds all the data GUI needs to access or change

### the PDFGuiControl object will be saved as a ddp file in the end

In [40]:
new_guicontrol = PDFGuiControl()

## Copy the Fitting object from ddp to the new PDFGuiControl

In [41]:
new_guicontrol.add(fit_demo.copy())

In [42]:
# dir(new_guicontrol)

## Using PDFGuiControl.fits to check the added Fitting object in the the new PDFGuiControl

In [43]:
new_guicontrol.fits

## Find the gr data list and sort it

In [44]:
subpath = '/Users/cheng-hunglin/Documents/Data_LDRD'
gr_dir = os.path.join(subpath, 'gr_data')
gr_list = glob.glob(gr_dir + '/*.gr')
gr_list.sort()

In [45]:
gr_serial = gr_list[:]

## Define fitrmax, fitrmin, qdamp, qbroad from the copied Fitting object, e.g., fit_demo.copy()

In [46]:
fitrmax = new_guicontrol.fits[0].datasets[0].fitrmax
fitrmin = new_guicontrol.fits[0].datasets[0].fitrmin
qdamp = new_guicontrol.fits[0].datasets[0].qdamp
qbroad = new_guicontrol.fits[0].datasets[0].qbroad
# new_guicontrol.fits[0].datasets[1].constraints.update(new_guicontrol.fits[0].datasets[0].constraints)

## Check fitting parameters

In [47]:
new_guicontrol.fits[0].parameters

{11: <diffpy.pdfgui.control.parameter.Parameter at 0x364fc9fd0>,
 12: <diffpy.pdfgui.control.parameter.Parameter at 0x364fca2d0>,
 91: <diffpy.pdfgui.control.parameter.Parameter at 0x364fcb750>,
 92: <diffpy.pdfgui.control.parameter.Parameter at 0x364fcb7d0>,
 131: <diffpy.pdfgui.control.parameter.Parameter at 0x364fc8050>,
 132: <diffpy.pdfgui.control.parameter.Parameter at 0x364fcbc50>,
 133: <diffpy.pdfgui.control.parameter.Parameter at 0x364fcbed0>,
 14: <diffpy.pdfgui.control.parameter.Parameter at 0x364fcbe50>,
 54: <diffpy.pdfgui.control.parameter.Parameter at 0x364fcbbd0>,
 94: <diffpy.pdfgui.control.parameter.Parameter at 0x364fcbdd0>,
 134: <diffpy.pdfgui.control.parameter.Parameter at 0x364fcbf50>,
 141: <diffpy.pdfgui.control.parameter.Parameter at 0x364fca050>,
 142: <diffpy.pdfgui.control.parameter.Parameter at 0x364fca450>,
 143: <diffpy.pdfgui.control.parameter.Parameter at 0x364fca4d0>,
 261: <diffpy.pdfgui.control.parameter.Parameter at 0x364fca550>,
 262: <diffpy.pdf

In [48]:
new_guicontrol.fits[0].parameters[1].fixed

False

In [49]:
new_guicontrol.fits[0].parameters[1].fixed = False

In [50]:
new_guicontrol.fits[0].parameters[1].fixed

False

In [51]:
## Fitting parameters of lattice parameter in a list
lat_par_list = [1 ,2 ,3, 4, 5, 6]

## Make the fitting tree in the new PDFGuiControl

In [52]:
for i in range(len(gr_serial)):

    ## Add new Fitting by copying the existed Fitting object
    new_guicontrol.add(fit_demo.copy())

    ## Rename the copied Fitting object
    new_guicontrol.fits[i+1].name = f'Fit 3 _3D_s{i:02d}'

    ## Load gr data from gr_list to the newly copied Fitting object
    new_guicontrol.loadDataset(new_guicontrol.fits[i+1], gr_list[i])

    ## Delete the old dataset in the newly copied Fitting object  
    new_guicontrol.fits[i+1].datasets.__delitem__(0)

    ## Update fitrmax, fitrmin, qdamp, qbroad in the loaded dataset
    new_guicontrol.fits[i+1].datasets[0].fitrmax = fitrmax
    new_guicontrol.fits[i+1].datasets[0].fitrmin = fitrmin
    new_guicontrol.fits[i+1].datasets[0].qdamp = qdamp
    new_guicontrol.fits[i+1].datasets[0].qbroad = qbroad

    ## Update the constraints for the loaded dataset, mainly for dscale, qdamp, qbroad if constrained
    new_guicontrol.fits[i+1].datasets[0].constraints.update(deepcopy(new_guicontrol.fits[0].datasets[0].constraints))

    ## Equal the amount of scale factor
    num_struc = len(new_guicontrol.fits[0].strucs)
    for j in range(num_struc):
        new_guicontrol.fits[i+1].strucs[j].setvar('pscale', 1/num_struc)
    # new_guicontrol.fits[i+1].parameters[100].setInitial(0.615)
    # new_guicontrol.fits[i+1].parameters[101].setInitial(0.615)
    
    new_guicontrol.fits[i+1].parameters[300].setInitial(0.5)
    
    ## Free lattice parameters for fitting
    for k in lat_par_list:
        new_guicontrol.fits[i+1].parameters[k].fixed = False
    

In [53]:
len(new_guicontrol.fits)

85

In [54]:
new_guicontrol.fits[i+1].strucs[1].getvar('pscal')

0.3333333333333333

In [55]:
new_guicontrol.fits[i+1].parameters[100].initialValue()

0.33

In [56]:
new_guicontrol.fits[1].datasets[0].constraints

{'dscale': <diffpy.pdfgui.control.constraint.Constraint at 0x394feea50>}

## Run all the fitting in the new PDFGuiControl

In [57]:
## The total # of Fitting object in the new PDFGuiControl will be len(gr_list) + 1
## because of the first one is from the pre-defined ddp and has no modification 
# for i in range(2):
# for i in tqdm(range(len(gr_serial)+1), desc =f'batch fitting with pdffit2'):
failed_list = []
for i in range(len(gr_serial)+1):
    
    clear_output(wait=True)
    
    data_name = new_guicontrol.fits[i].datasets.keys()[0]
    print(f'{i = }\n')
    print(f'Data {data_name[:28]} fitting is on going\n')
    
    try:
        new_guicontrol.fits[i].run()

        print('\n')
        print('Update refined results to initial for the current dataset\n')
        for key, item in new_guicontrol.fits[i].parameters.items():
            refined = new_guicontrol.fits[i].getData(key)
            new_guicontrol.fits[i].parameters[key].setInitial(refined)

        # print('Update refined results to initial for the next dataset\n')
        # try:
        #     for key, item in new_guicontrol.fits[i].parameters.items():
        #         refined = new_guicontrol.fits[i].getData(key)
        #         new_guicontrol.fits[i+1].parameters[key].setInitial(refined)
        # except IndexError:
        #     pass
        
    except calculationError:
        new_guicontrol.fits[i].stop()
        failed_list.append(data_name)
        print(f'Data {data_name[:28]} fitting fialed\n')
        
        # print('Update refined resutls to initial for the next dataset from the previous one\n')
        # try:
        #     for key, item in new_guicontrol.fits[i-1].parameters.items():
        #         refined = new_guicontrol.fits[i-1].getData(key)
        #         new_guicontrol.fits[i+1].parameters[key].setInitial(refined)
        # except IndexError:
        #     pass
        
        tqdm_sleep_sec(30, message=f'fitting failed: {i = }, {data_name}')
    
    print('\n')
    time.sleep(3)


i = 0

Data Cs_078_Br_092_Pb_000_OLA_020 fitting is on going


******************************************************************************
*                      P D F F I T   Version   1.5.1                         *
*                                              2025-02-07                    *
* -------------------------------------------------------------------------- *
* (c) 1998-2007 Trustees of the Michigan State University.                   *
* (c) 2008-2025 Trustees of the Columbia University                          *
*               in the city of New York.                                     *
*                                                                            *
* Authors:                                                                   *
*     Thomas Proffen        -   Email: tproffen@lanl.gov                     *
*     Jacques Bloch         -   Email: bloch@pa.msu.edu                      *
*     Christopher Farrow    -   Email: clf2121@columbia.edu         

KeyboardInterrupt: 

In [ ]:
i

## Save the new PDFGuiControl as a ddp file

In [ ]:
subpath = '/Users/cheng-hunglin/Documents/Data_LDRD'
fn = 'Cs_x-Pb_y-B_z_serial_all5-1.ddp'
full_fn = os.path.join(subpath, fn)

new_guicontrol.save(projfile=full_fn)

## Plot rw for all fittins

In [ ]:
rw_list = []
for i in range(len(gr_serial)+1):
    rw = new_guicontrol.fits[i].getData('rw')
    rw_list.append(rw)

In [ ]:
len(rw_list)

In [ ]:
idx = np.arange(-1, 84, 1)

In [ ]:
idx.shape

In [ ]:
f = plt.figure(figsize=(15,5))
ax = f.gca()
ax.plot(idx, rw_list, 'o--')
ax.set_ylabel('rw')

## Re fitting

In [58]:
fn = '/Users/cheng-hunglin/Documents/Data_LDRD/Cs_x-Pb_y-B_z_serial_all5-1.ddp'
ddp_01 = LoadProject(fn)

In [59]:
len(ddp_01.getFits())

85

In [60]:
ddp_01.getFits()[0].datasets.keys()[0]

'Cs_078_Br_092_Pb_000_OLA_020_PF_016_Tol_948_20250608-191128_dc413d_scattering-1_mean_q.gr'

In [61]:
ddp_01.getFits()[0].getData(1)

7.970430408803134

In [62]:
# ddp_01.getFits()[0].parameters[1].fixed = False

In [63]:
failed_list = []
for i in range(len(ddp_01.getFits())):
    
    clear_output(wait=True)
    
    data_name = ddp_01.getFits()[i].datasets.keys()[0]
    print(f'{i = }\n')
    print(f'Data {data_name[:28]} fitting is on going\n')

    ## Free delta2 for fitting
    for k in [200, 201, 202]:
        ddp_01.getFits()[i].parameters[k].fixed = False
    
    try:
        ddp_01.getFits()[i].run()

        print('\n')
        print('Update refined results to initial for the current dataset\n')
        for key, item in ddp_01.getFits()[i].parameters.items():
            refined = ddp_01.getFits()[i].getData(key)
            ddp_01.getFits()[i].parameters[key].setInitial(refined)

        # print('Update refined results to initial for the next dataset\n')
        # try:
        #     for key, item in new_guicontrol.fits[i].parameters.items():
        #         refined = new_guicontrol.fits[i].getData(key)
        #         new_guicontrol.fits[i+1].parameters[key].setInitial(refined)
        # except IndexError:
        #     pass
        
    except calculationError:
        ddp_01.getFits()[i].stop()
        failed_list.append(data_name)
        print(f'Data {data_name[:28]} fitting fialed\n')
        
        # print('Update refined resutls to initial for the next dataset from the previous one\n')
        # try:
        #     for key, item in new_guicontrol.fits[i-1].parameters.items():
        #         refined = new_guicontrol.fits[i-1].getData(key)
        #         new_guicontrol.fits[i+1].parameters[key].setInitial(refined)
        # except IndexError:
        #     pass
        
        tqdm_sleep_sec(30, message=f'fitting failed: {i = }, {data_name}')
    
    print('\n')
    time.sleep(3)

i = 84

Data Cs_180_Br_040_Pb_020_PF_020_ fitting is on going


******************************************************************************
*                      P D F F I T   Version   1.5.1                         *
*                                              2025-02-07                    *
* -------------------------------------------------------------------------- *
* (c) 1998-2007 Trustees of the Michigan State University.                   *
* (c) 2008-2025 Trustees of the Columbia University                          *
*               in the city of New York.                                     *
*                                                                            *
* Authors:                                                                   *
*     Thomas Proffen        -   Email: tproffen@lanl.gov                     *
*     Jacques Bloch         -   Email: bloch@pa.msu.edu                      *
*     Christopher Farrow    -   Email: clf2121@columbia.edu        

In [64]:
ddp_01.getFits()[i].stop()

In [65]:
subpath = '/Users/cheng-hunglin/Documents/Data_LDRD'
fn = 'Cs_x-Pb_y-B_z_serial_all5-2.ddp'
full_fn = os.path.join(subpath, fn)

ddp_01.save(full_fn)

In [66]:
failed_list

['Cs_010_Br_136_Pb_093_OLA_000_PF_020_Tol_1195_20250608-173020_a5e8e9_scattering-1_mean_q.gr']